<a href="https://colab.research.google.com/github/Swayamprakashpatel/COVID19_Clinical-Trial-Text-Mining/blob/master/AutoDock_Vina_SKP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Find Coordinates (Auto-Restart Enabled)
!pip install biopython -q

import numpy as np
from Bio.PDB import PDBParser
from google.colab import files
import os
import sys
import time

# Step 1: Install Biopython (if not already installed)
try:
    import Bio
except ImportError:
    print("Installing Biopython...")
    !pip install biopython -q
    print("Biopython installed!")

def calculate_grid_center(pdb_filename):
    """
    Parses the uploaded PDB file and finds the center of the CDK2-Dinaciclib pocket.
    It looks for specific anchor residues defined in Martin et al. (2013).
    """
    parser = PDBParser(QUIET=True)
    try:
        structure = parser.get_structure('target', pdb_filename)
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return

    # Critical Anchors for Dinaciclib Binding (Residue IDs)
    # Leu83 (Hinge), Lys33 (Catalytic), Phe80 (Gatekeeper), Asp145 (DFG)
    target_residues = [83, 33, 80, 145]

    atom_coords = []
    found_residues = []

    print(f"\nScanning {pdb_filename} for CDK2 active site anchors...")

    # Iterate through ALL chains and residues
    for model in structure:
        for chain in model:
            for residue in chain:
                # Check if residue number matches our targets
                if residue.id[1] in target_residues:
                    # We use Alpha Carbon (CA) to get the reliable backbone center
                    if 'CA' in residue:
                        atom_coords.append(residue['CA'].get_coord())
                        res_name = f"{residue.resname}{residue.id[1]}"
                        if res_name not in found_residues:
                            found_residues.append(res_name)

    # --- RESULTS ---
    if not atom_coords:
        print("❌ Error: Could not find the specific CDK2 anchor residues (Leu83, Lys33, etc).")
        print("   Check if your PDB file is standard numbering.")
    else:
        center = np.mean(atom_coords, axis=0)

        print("-" * 50)
        print(f"✅ SUCCESS! Found {len(found_residues)} anchor residues:")
        print(f"   {', '.join(found_residues)}")
        print("-" * 50)
        print("👇 COPY THESE COORDINATES FOR YOUR DOCKING CONFIG 👇")
        print(f"center_x = {center[0]:.3f}")
        print(f"center_y = {center[1]:.3f}")
        print(f"center_z = {center[2]:.3f}")
        print("-" * 50)
        print(f"Recommended Box Size: 22 x 22 x 22")

# --- MAIN EXECUTION ---
print("Please upload your PDB file (e.g., 4KD1.pdb)...")
uploaded = files.upload()

for filename in uploaded.keys():
    calculate_grid_center(filename)

    # 1. DELETE THE FILE
    if os.path.exists(filename):
        os.remove(filename)
        print(f"\n🗑️ File '{filename}' has been deleted to clean workspace.")

# 2. RESTART RUNTIME
print("\n⚠️ SYSTEM NOTIFICATION: Restarting Runtime in 10 seconds...")
print("⏳ Please copy the coordinates above quickly!")
time.sleep(10) # Gives you 10 seconds to read the output
print("🔄 Restarting now...")
os.kill(os.getpid(), 9)

In [ ]:
# @title Basic Installation (Run and wait for runtime restart/crash then run next cells)
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# @title 🛠️ Basic Setup, Receptor & Ligand Preparation
import os
import shutil
import glob
import subprocess
import pandas as pd
from pathlib import Path
from google.colab import files

# =============================================================================
# 1. INSTALLATION & SETUP (Dependencies)
# =============================================================================
print("⚙️ Installing dependencies (This may take a few minutes)...")

# Install Conda (Required for MGLTools)
try:
    import condacolab
    condacolab.check()
except ImportError:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Install Bioinformatics Tools
!pip install -q pdb2pqr
!conda install -q -c bioconda mgltools openbabel

# Setup Directories
base_path = Path("/content/Docking")
os.makedirs(base_path, exist_ok=True)
singlepath = base_path / "single-dock"
os.makedirs(singlepath, exist_ok=True)
# Creating the ligands path as per original script requirement
ligandpath = base_path / "ligands"
os.makedirs(ligandpath, exist_ok=True)

local_path = base_path # Standardize path variable

# Install AutoDock Vina 1.2.7
print("⬇️ Setting up AutoDock Vina...")
%cd {base_path}
if not os.path.exists("vina_1.2.7_linux_x86_64"):
    !wget --no-check-certificate -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.7/vina_1.2.7_linux_x86_64
    !chmod +x vina_1.2.7_linux_x86_64

# Set alias (Optional helper)
%alias vina '{base_path}'/vina_1.2.7_linux_x86_64

# =============================================================================
# 2. FILE UPLOADS (Receptor & Ligands)
# =============================================================================
print("\n📂 UPLOAD SECTION")

# A. Upload Receptor (PDB)
print("👇 Please upload your RECEPTOR PDB file (e.g., 1HSG.pdb):")
uploaded_pdb = files.upload()
if not uploaded_pdb:
    raise ValueError("No PDB file uploaded!")
protein = list(uploaded_pdb.keys())[0]

# B. Upload Ligands (CSV)
print("👇 Please upload your LIGANDS CSV file (e.g., AI_Generated_Leads.csv):")
# Check if CSV already exists to avoid re-uploading loop if rerunning
existing_csv = glob.glob(str(base_path / "*.csv"))
if existing_csv:
    csv_name = existing_csv[0]
    print(f"   ✅ Found existing CSV: {csv_name}")
else:
    uploaded_csv = files.upload()
    if not uploaded_csv:
        raise ValueError("No CSV file uploaded!")
    csv_name = list(uploaded_csv.keys())[0]

# =============================================================================
# 3. RECEPTOR PREPARATION
# =============================================================================
print(f"\n🧬 Processing Receptor: {protein}")

# Cleaning Protocol
output_name = f"{os.path.splitext(protein)[0]}_prot.pdb"
output_path = singlepath / output_name

with open(output_path, "w") as g:
    with open(protein, 'r') as f:
        for line in f:
            row = line.split()
            if not row: continue
            if row[0] == "ATOM":
                g.write(line)
            elif row[0] == "TER":
                g.write("TER\n")
        g.write("END")

print(f"   ✅ Cleaned PDB saved to: {output_path}")

# PDBQT Preparation
protein_id = os.path.splitext(protein)[0]
cleaned_pdb = str(output_path)
output_pqr = str(singlepath / f"{protein_id}_prot.pqr")
output_pdbqt = str(singlepath / f"{protein_id}_prot.pdbqt")

# Running commands quietly (> /dev/null) to keep output clean
print("   running prepare_receptor4 (Hydrogens)...")
!prepare_receptor4.py -r "{cleaned_pdb}" -o "{output_pdbqt}" -A hydrogens -U nphs_lps -v > /dev/null 2>&1

print("   running pdb2pqr (Amber FF)...")
!pdb2pqr --ff=AMBER --keep-chain --with-ph=7.4 "{cleaned_pdb}" "{output_pqr}" > /dev/null 2>&1

print("   running prepare_receptor4 (Final Charges)...")
!prepare_receptor4.py -r "{output_pqr}" -o "{output_pdbqt}" -C -U nphs_lps -v > /dev/null 2>&1

print(f"✅ Receptor Preparation Complete: {output_pdbqt}")

# =============================================================================
# 4. LIGAND PREPARATION (BATCH)
# =============================================================================
df = pd.read_csv(csv_name)

# LIMITING TO FIRST 2 FOR DEMO (As per your code)
# Comment out the next line to process the full file
df = df.head(2)

df = df.dropna(subset=['Compound_ID', 'SMILES'])

print(f"\n🚀 Processing {len(df)} ligands through the 3-Step Expert Pipeline...")

# Move to base path to ensure local file generation works for MGLTools
%cd {base_path}

output_dir = singlepath # Output to single-dock

for index, row in df.iterrows():
    # Sanitize inputs
    lead_id = str(row['Compound_ID']).replace(" ", "_").replace("(", "").replace(")", "")
    smiles = str(row['SMILES']).strip()

    if not smiles or smiles.lower() == "nan":
        print(f"⚠️ Skipping index {index}: Missing SMILES.")
        continue

    # Local temp filenames
    temp_smiles = f"{lead_id}.smiles"
    temp_mol2 = f"{lead_id}.mol2"
    temp_pdbqt = f"{lead_id}.pdbqt"

    # Save SMILES locally
    with open(temp_smiles, "w") as f:
        f.write(smiles)

    print(f"\n🧪 Lead {index+1}/{len(df)}: {lead_id}")

    # Step 1: Standard
    dest_std = output_dir / f"{lead_id}_std.pdbqt"
    !obabel "{temp_smiles}" -O "{temp_mol2}" --gen3d best -p 7.4 --canonical > /dev/null 2>&1
    !prepare_ligand4.py -l "{temp_mol2}" -o "{temp_pdbqt}" -U nphs_lps > /dev/null 2>&1
    if os.path.exists(temp_pdbqt):
        shutil.move(temp_pdbqt, dest_std)
    if os.path.exists(temp_mol2): os.remove(temp_mol2)

    # Step 2: GAFF Minimized
    dest_min = output_dir / f"{lead_id}_minimized.pdbqt"
    !obabel "{temp_smiles}" -O "{temp_mol2}" --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd > /dev/null 2>&1
    !prepare_ligand4.py -l "{temp_mol2}" -o "{temp_pdbqt}" -U nphs_lps > /dev/null 2>&1
    if os.path.exists(temp_pdbqt):
        shutil.move(temp_pdbqt, dest_min)
    if os.path.exists(temp_mol2): os.remove(temp_mol2)

    # Step 3: Weighted Rotor
    dest_rotor = output_dir / f"{lead_id}_rotor.pdbqt"
    !obabel "{temp_smiles}" -O "{temp_mol2}" --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF > /dev/null 2>&1
    !prepare_ligand4.py -l "{temp_mol2}" -o "{temp_pdbqt}" -U nphs_lps > /dev/null 2>&1
    if os.path.exists(temp_pdbqt):
        shutil.move(temp_pdbqt, dest_rotor)

    # Cleanup
    if os.path.exists(temp_smiles): os.remove(temp_smiles)
    if os.path.exists(temp_mol2): os.remove(temp_mol2)

print(f"\n🎉 DONE: {len(df)} ligands prepared in: {output_dir}")

In [ ]:
# @title 🚀 Grid Visualization & Analysis
import os
import glob
import subprocess
import pandas as pd
import sys
from pathlib import Path
import ipywidgets
from ipywidgets import interact, fixed

# =============================================================================
# 1. SETUP & VISUALIZATION
# =============================================================================
# Ensure dependencies
try:
    import py3Dmol
except ImportError:
    !{sys.executable} -m pip install py3Dmol kora rdkit -q
    import py3Dmol

# Define Paths
base_path = Path('/content/Docking')
docking_dir = base_path / "single-dock"
vina_binary = base_path / "vina_1.2.7_linux_x86_64"

# Safety Check: Ensure binary is executable
!chmod +x "{vina_binary}"

%cd {docking_dir}

# Auto-detect the cleaned protein file for visualization
pdb_files = glob.glob("*_prot.pdb")
if not pdb_files:
    raise FileNotFoundError("No Receptor PDB found in single-dock. Run the Prep cell first.")
protein_to_visualize = str(docking_dir / pdb_files[0])

# Visualization Function
def viewprotgrid(prot_PDBfile, resids, bxi, byi, bzi, bxf, byf, bzf):
    mol_view = py3Dmol.view(width=800, height=500)
    with open(prot_PDBfile, 'r') as f:
        mol1 = f.read()
    mol_view.addModel(mol1, 'pdb')
    mol_view.setStyle({'cartoon': {'color': 'spectrum'}})
    mol_view.addStyle({'resi': resids}, {'stick': {'colorscheme': 'greenCarbon'}})

    # Draw the Box
    mol_view.addBox({
        'center': {'x': bxi, 'y': byi, 'z': bzi},
        'dimensions': {'w': bxf, 'h': byf, 'd': bzf},
        'color': 'blue', 'opacity': 0.6
    })

    mol_view.setBackgroundColor('white')
    mol_view.zoomTo()
    mol_view.show()

print(f"📊 Visualizing Receptor: {os.path.basename(protein_to_visualize)}")
# Note: Interactivity works, but execution proceeds immediately to docking below.
interact(viewprotgrid,
         prot_PDBfile = fixed(protein_to_visualize),
         resids = fixed(['32,47,82-84']),
         bxi=ipywidgets.IntSlider(min=-100, max=100, step=1, value=15, description='Center X'),
         byi=ipywidgets.IntSlider(min=-100, max=100, step=1, value=29, description='Center Y'),
         bzi=ipywidgets.IntSlider(min=-100, max=100, step=1, value=4,  description='Center Z'),
         bxf=ipywidgets.IntSlider(min=1, max=50, step=1, value=20,    description='Size X'),
         byf=ipywidgets.IntSlider(min=1, max=50, step=1, value=20,    description='Size Y'),
         bzf=ipywidgets.IntSlider(min=1, max=50, step=1, value=20,    description='Size Z'))


In [ ]:
# @title 🚀 Run Multi-Ligand Docking & Analysis
import os
import glob
import subprocess
import pandas as pd
from pathlib import Path

# =============================================================================
# A. CONFIGURATION (Enter your coordinates here)
# =============================================================================
center_x = 55 # @param {type:"number"}
center_y = 77 # @param {type:"number"}
center_z = 15 # @param {type:"number"}
size_x = 20 # @param {type:"number"}
size_y = 20 # @param {type:"number"}
size_z = 20 # @param {type:"number"}
force_docking_rerun = True # @param {type:"boolean"}

# Setup Paths & Binary
base_path = Path('/content/Docking')
docking_dir = base_path / "single-dock"
vina_binary = base_path / "vina_1.2.7_linux_x86_64"
!chmod +x "{vina_binary}"

%cd {docking_dir}

# 1. Create Config File
pdbqt_files = glob.glob("*_prot.pdbqt")
if not pdbqt_files: raise FileNotFoundError("Receptor PDBQT not found.")
receptor = pdbqt_files[0]

with open("config_template", "w") as f:
    f.write(f"receptor = {receptor}\n")
    f.write(f"center_x = {center_x}\ncenter_y = {center_y}\ncenter_z = {center_z}\n")
    f.write(f"size_x = {size_x}\nsize_y = {size_y}\nsize_z = {size_z}\n")
    f.write("exhaustiveness = 8\n")

print(f"✅ Grid Configured based on {receptor}")
if force_docking_rerun: print("⚠️ Force Rerun Active: Overwriting previous logs.")

# =============================================================================
# B. DOCKING LOOP
# =============================================================================
ligands = glob.glob("*_std.pdbqt") + glob.glob("*_minimized.pdbqt") + glob.glob("*_rotor.pdbqt")
print(f"\n🧬 Starting Vina Docking for {len(ligands)} ligands...")

for ligand in ligands:
    name = os.path.splitext(ligand)[0]
    log_file = f"{name}.log"
    out_file = f"{name}_out.pdbqt"

    if os.path.exists(log_file) and not force_docking_rerun:
        print(f"⏩ Skipping {name} (Log exists)")
        continue

    print(f"▶️ Docking {ligand}...")

    # Vina 1.2.7 Command
    cmd = [str(vina_binary), "--config", "config_template", "--ligand", ligand, "--out", out_file]

    try:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            with open(log_file, 'w') as f: f.write(result.stdout)
            # Quick score check
            for line in result.stdout.split('\n'):
                if "   1   " in line:
                    print(f"   ✅ Score: {line.split()[1]} kcal/mol")
                    break
        else:
            print(f"   ❌ Error: {result.stderr}")
    except Exception as e:
        print(f"   ⚠️ System Error: {e}")

print("\n🎉 DOCKING COMPLETE.")

# =============================================================================
# C. RESULTS ANALYSIS
# =============================================================================
print("\n📊 TABULATING RESULTS...")
results_data = []
for log in glob.glob("*.log"):
    file_name = os.path.basename(log)
    parts = file_name.replace('.log', '').split('_')

    # Determine Protocol
    if "std" in parts: protocol = "Standard"
    elif "minimized" in parts: protocol = "GAFF_Minimized"
    elif "rotor" in parts: protocol = "Weighted_Rotor"
    else: protocol = "Unknown"

    lead_id = "_".join(parts[:-1])

    # Get Score
    with open(log, 'r') as f:
        for line in f:
            if "   1   " in line:
                try: results_data.append({'Lead_ID': lead_id, 'Protocol': protocol, 'Affinity': float(line.split()[1])})
                except: pass
                break

if results_data:
    df = pd.DataFrame(results_data).sort_values(by='Affinity')
    csv_out = base_path / "Final_Docking_Results.csv"
    df.to_csv(csv_out, index=False)
    from IPython.display import display
    print(f"🏆 Top 10 Candidates (Full table saved to {csv_out})")
    display(df.head(10))
else:
    print("⚠️ No results found.")

In [ ]:
# @title 🔄 Visualization
import glob
import os
import subprocess
from pathlib import Path

# Setup Paths
base_path = Path('/content/Docking')
docking_dir = base_path / "single-dock"
%cd {docking_dir}

# Find all output files
docking_results = glob.glob("*_out.pdbqt")

print(f"🔄 Converting {len(docking_results)} docking results to PDB format for visualization...")

conversion_count = 0
for pdbqt_file in docking_results:
    name_base = os.path.splitext(pdbqt_file)[0] # e.g. Lead_20_std_out
    pdb_output = f"{name_base}.pdb"

    # We use OpenBabel to convert.
    # -f 1 -l 1 means "Only take the 1st model" (The best scoring pose)
    cmd = f"obabel {pdbqt_file} -O {pdb_output} -f 1 -l 1"

    try:
        subprocess.run(cmd, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        conversion_count += 1
    except Exception as e:
        print(f"❌ Failed to convert {pdbqt_file}")

print(f"✅ Successfully converted {conversion_count} files. Ready for visualization!")


# @title 2. 🧬 Interactive Docking Dashboard
import ipywidgets
from ipywidgets import interact, Layout
import py3Dmol
import glob
import os

# 1. Setup Data for Dropdowns
# Get list of unique Lead IDs (e.g., Lead_1, Lead_20)
all_files = glob.glob("*_out.pdb")
leads = sorted(list(set([f.split('_')[0] + "_" + f.split('_')[1] for f in all_files if "Lead" in f])))

# Protocols available
protocols = ['Standard', 'GAFF_Minimized', 'Weighted_Rotor']

# Find the Protein File automatically
prot_files = glob.glob("*_prot.pdb")
protein_file = prot_files[0] if prot_files else None

# 2. Define the Visualization Function
def view_docking_result(lead_id, protocol, show_surface):

    # Construct filename based on selection
    # e.g., Lead_20 + Standard -> Lead_20_std_out.pdb
    suffix_map = {
        'Standard': '_std_out.pdb',
        'GAFF_Minimized': '_minimized_out.pdb',
        'Weighted_Rotor': '_rotor_out.pdb'
    }

    filename = f"{lead_id}{suffix_map[protocol]}"

    if not os.path.exists(filename):
        print(f"⚠️ File not found: {filename}")
        print("Try a different protocol for this lead.")
        return

    print(f"🔬 Viewing: {lead_id} | Protocol: {protocol}")

    # Initialize Viewer
    view = py3Dmol.view(width=800, height=600)

    # A. Load Protein (Rainbow Cartoon)
    if protein_file:
        with open(protein_file, 'r') as f:
            prot_data = f.read()
        view.addModel(prot_data, 'pdb')
        view.setStyle({'model': 0}, {'cartoon': {'color': 'spectrum'}})
        # Highlight Active Site (Optional, same residues as before)
        view.addStyle({'resi': ['32','47','82','83','84']}, {'stick': {'colorscheme': 'lightgreyCarbon'}})

    # B. Load Docked Ligand (Green Sticks)
    with open(filename, 'r') as f:
        ligand_data = f.read()
    view.addModel(ligand_data, 'pdb')
    view.setStyle({'model': 1}, {'stick': {'colorscheme': 'greenCarbon'}})

    # C. Optional: Add Surface
    if show_surface:
        view.addSurface(py3Dmol.SAS, {'opacity': 0.7, 'color': 'white'}, {'model': 0})

    view.zoomTo({'model': 1}) # Zoom to the ligand, not the whole protein
    view.setBackgroundColor('white')
    view.show()

# 3. Launch the Dashboard
if not leads:
    print("❌ No converted PDB files found. Did you run the conversion cell above?")
else:
    interact(view_docking_result,
             lead_id = ipywidgets.Dropdown(options=leads, description='Lead ID:'),
             protocol = ipywidgets.Dropdown(options=protocols, description='Protocol:'),
             show_surface = ipywidgets.Checkbox(value=False, description='Show Pocket Surface'))